In [1]:
import sys
sys.executable

'd:\\1.VINA\\ai_project\\milvus_ai_local\\.venv1\\Scripts\\python.exe'

In [2]:
import pymilvus
print(pymilvus.__version__)

2.6.5


In [14]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(("10.50.171.40", 19530))
print("Connection result:", result)

Connection result: 0


In [3]:
from pymilvus import connections, utility

connections.connect(
    alias="default",
    host="10.50.171.40",
    port=19530
)

print("Milvus version:", utility.get_server_version())

Milvus version: 2.6.7


In [4]:
#Read Excel File for sample first

import pandas as pd


df = pd.read_excel(r"D:\\1.VINA\\ai_project\\sample_dataset.xlsx")

df.head()

,Ticket Number,Ticket Summary,Type Ticket,Category,Sub Category,New Category,Month,Created By,Department,Assign Name,Quarter,Troubleshoot Step
0,11549012025,Assign Acc Accrued New Transaction on Dec (Bal...,Yonsuite,Custom Report Yonsuite,Custom Existing Report Yonsuite,Custom Report Yonsuite,January,Dian Annisa Gustamal,FA,Dea Aldiani,Q1,1. Add new COAs in report item \n2. Recalculat...
1,11668022025,missing JIT menu,JIT,Installation JIT,Installation JIT,Installation JIT,February,Pissa Wigiani,PRODUKSI,Dea Aldiani,Q1,1. Check and configure the system date and tim...
2,11366012025,Installation JIT,JIT,Installation JIT,Installation JIT,Installation JIT,January,Kiki Nur Alif,LABORAT,Dea Aldiani,Q1,1. Check and configure the system date and tim...
3,11463012025,installation,JIT,Installation JIT,Installation JIT,Installation JIT,January,Ilman Maulana Firmansyah,PRODSCAN,Dea Aldiani,Q1,1. Check and configure the system date and tim...
4,11487012025,LUPA PASSWORD ORANGE,Orange,Troubleshooting Orange,OTHER ORANGE,Account Access OrangE,January,Enok Nurbayasari,PRODUKSI,Ervina Maharani,Q1,Update new password using upload template


In [5]:
df.drop(columns=['Sub Category', 'Quarter'], inplace=True)
df.head()

,Ticket Number,Ticket Summary,Type Ticket,Category,New Category,Month,Created By,Department,Assign Name,Troubleshoot Step
0,11549012025,Assign Acc Accrued New Transaction on Dec (Bal...,Yonsuite,Custom Report Yonsuite,Custom Report Yonsuite,January,Dian Annisa Gustamal,FA,Dea Aldiani,1. Add new COAs in report item \n2. Recalculat...
1,11668022025,missing JIT menu,JIT,Installation JIT,Installation JIT,February,Pissa Wigiani,PRODUKSI,Dea Aldiani,1. Check and configure the system date and tim...
2,11366012025,Installation JIT,JIT,Installation JIT,Installation JIT,January,Kiki Nur Alif,LABORAT,Dea Aldiani,1. Check and configure the system date and tim...
3,11463012025,installation,JIT,Installation JIT,Installation JIT,January,Ilman Maulana Firmansyah,PRODSCAN,Dea Aldiani,1. Check and configure the system date and tim...
4,11487012025,LUPA PASSWORD ORANGE,Orange,Troubleshooting Orange,Account Access OrangE,January,Enok Nurbayasari,PRODUKSI,Ervina Maharani,Update new password using upload template


In [6]:
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    utility
)

collection_name = "kb_docs"

if not utility.has_collection(collection_name):
    fields = [
    FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=True
    ),
    FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=384
    ),
    FieldSchema(
        name="ticket_number",
        dtype=DataType.VARCHAR,
        max_length=12
    ),
    FieldSchema(
        name="ticket_summary",
        dtype=DataType.VARCHAR,
        max_length=1024
    ),
    FieldSchema(
        name="type_ticket",
        dtype=DataType.VARCHAR,
        max_length=64
    ),
    FieldSchema(
        name="category",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="new_category",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="department",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="assign_name",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="troubleshoot_step",
        dtype=DataType.VARCHAR,
        max_length=4096
    ),
    ]

    schema = CollectionSchema(
        fields=fields,
        description="Apollo Helpdesk Knowledge Base"
    )

    collection = Collection(
            name=collection_name,
            schema=schema
    )

    print("Collection created:", collection_name)
else:
    collection = Collection(collection_name)
    print("Collection already exists:", collection_name)

Collection already exists: kb_docs


In [21]:
collection.create_index(
    field_name="embedding",
    index_params={
        "metric_type": "IP",
        "index_type": "HNSW",
        "params": {"M": 8, "efConstruction": 64}
    }
)

Status(code=0, message=)

In [ ]:
collection.load()

In [8]:
df["embed_text"] = (
    "Problem: " + df["Ticket Summary"].fillna("") +
    "\nSolution: " + df["Troubleshoot Step"].fillna("")
)

df.head()

,Ticket Number,Ticket Summary,Type Ticket,Category,New Category,Month,Created By,Department,Assign Name,Troubleshoot Step,embed_text
0,11549012025,Assign Acc Accrued New Transaction on Dec (Bal...,Yonsuite,Custom Report Yonsuite,Custom Report Yonsuite,January,Dian Annisa Gustamal,FA,Dea Aldiani,1. Add new COAs in report item \n2. Recalculat...,Problem: Assign Acc Accrued New Transaction on...
1,11668022025,missing JIT menu,JIT,Installation JIT,Installation JIT,February,Pissa Wigiani,PRODUKSI,Dea Aldiani,1. Check and configure the system date and tim...,Problem: missing JIT menu\nSolution: 1. Check ...
2,11366012025,Installation JIT,JIT,Installation JIT,Installation JIT,January,Kiki Nur Alif,LABORAT,Dea Aldiani,1. Check and configure the system date and tim...,Problem: Installation JIT\nSolution: 1. Check ...
3,11463012025,installation,JIT,Installation JIT,Installation JIT,January,Ilman Maulana Firmansyah,PRODSCAN,Dea Aldiani,1. Check and configure the system date and tim...,Problem: installation\nSolution: 1. Check and ...
4,11487012025,LUPA PASSWORD ORANGE,Orange,Troubleshooting Orange,Account Access OrangE,January,Enok Nurbayasari,PRODUKSI,Ervina Maharani,Update new password using upload template,Problem: LUPA PASSWORD ORANGE\nSolution: Updat...


In [11]:
import torch
print(torch.__file__)

d:\1.VINA\ai_project\milvus_ai_local\.venv1\Lib\site-packages\torch\__init__.py


In [10]:
from sentence_transformers import SentenceTransformer

# Create the embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

AttributeError: module 'torch.utils._pytree' has no attribute 'register_pytree_node'

In [ ]:
texts = df["embed_text"].tolist()
embeddings = model.encode(texts)

In [ ]:
data_to_insert = [
    embeddings.tolist(),
    df["Ticket Number"].astype(str).tolist(),
    df["Ticket Summary"].tolist(),
    df["Troubleshoot Step"].tolist(),
    df["Type Ticket"].tolist(),
    df["Category"].tolist(),
    df["New Category"].tolist(),
    df["Department"].tolist(),
    df["Assign Name"].tolist(),
    df["Month"].tolist()
]

collection.insert(data_to_insert)
collection.flush()